## This notebook serves the purpose of performing the general linear modell on the respective data. 

### 0. Introduction

First we will need to load the data. We are now dealing with the preprocessed data. The preprocessing has been done with using [fMRI-prep](https://fmriprep.org/en/stable/). fMRIprep describes itself as such "(...)It performs basic processing steps (coregistration, normalization, unwarping, noise component extraction, segmentation, skullstripping etc.) providing outputs that can be easily submitted to a variety of group level analyses, including task-based or resting-state fMRI, graph theory measures, surface or volume-based statistics, etc."

Thus, we will now perform our first level model. The predictors are the respective categories of the stimuli. The dependent variable is the brain activity in a given voxel.
So lets start by loading the data! For now we are dealing with the first five runs of subject one, for the first session in the imagery task.

In [1]:
import os
os.chdir('/mnt/c/Users/janos/git/sub01-ses-01-imagery')

We can now take a look at all the files we have. In total, we have five runs. For each run, we have .json file, a nii.gz file and a confounds.tsv file. In the Dataset_Exploration notebook we had a similar structure. But instead of the events.tsv file, we now have the confounds file. The confound file contain the regressors of no interest. Those are variables, that we should explicitly exclude from our first level model and are computed during the preprocessing. We also see that we have preprocessed data, because the respective file names carry the additional "preproc". 

In [2]:
ls

sub-01_ses-imagery01_task-imagery_run-01_desc-preproc_bold.json*
sub-01_ses-imagery01_task-imagery_run-01_desc-preproc_bold.nii.gz*
sub-01_ses-imagery01_task-imagery_run-01_desc-preproc_confounds.tsv*
sub-01_ses-imagery01_task-imagery_run-02_desc-preproc_bold.json*
sub-01_ses-imagery01_task-imagery_run-02_desc-preproc_bold.nii.gz*
sub-01_ses-imagery01_task-imagery_run-02_desc-preproc_confounds.tsv*
sub-01_ses-imagery01_task-imagery_run-03_desc-preproc_bold.json*
sub-01_ses-imagery01_task-imagery_run-03_desc-preproc_bold.nii.gz*
sub-01_ses-imagery01_task-imagery_run-03_desc-preproc_confounds.tsv*
sub-01_ses-imagery01_task-imagery_run-04_desc-preproc_bold.json*
sub-01_ses-imagery01_task-imagery_run-04_desc-preproc_bold.nii.gz*
sub-01_ses-imagery01_task-imagery_run-04_desc-preproc_confounds.tsv*
sub-01_ses-imagery01_task-imagery_run-05_desc-preproc_bold.json*
sub-01_ses-imagery01_task-imagery_run-05_desc-preproc_bold.nii.gz*
sub-01_ses-imagery01_task-imagery_run-05_desc-preproc_confounds.

We can now load one sample confounds.tsv file, to see what regressors of no interest are

In [3]:
import pandas as pd
events=pd.read_csv('sub-01_ses-imagery01_task-imagery_run-01_desc-preproc_confounds.tsv',sep='\t')
events

,global_signal,global_signal_derivative1,global_signal_derivative1_power2,global_signal_power2,csf,csf_derivative1,csf_power2,csf_derivative1_power2,white_matter,white_matter_derivative1,...,rot_z_derivative1,rot_z_power2,rot_z_derivative1_power2,motion_outlier00,motion_outlier01,motion_outlier02,motion_outlier03,motion_outlier04,motion_outlier05,motion_outlier06
0,554.498550,NaN,NaN,307468.641921,826.870090,NaN,683714.146374,NaN,551.701398,NaN,...,NaN,0.000031,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,552.545617,-1.952933,3.813947,305306.659011,815.862929,-11.007161,665632.319586,121.157593,551.526335,-0.175063,...,-0.000709,0.000040,5.032484e-07,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,550.823228,-1.722389,2.966623,303406.228820,810.248197,-5.614732,656502.140689,31.525220,551.675734,0.149399,...,-0.000045,0.000040,2.061160e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,548.659178,-2.164050,4.683113,301026.893801,807.405975,-2.842222,651904.408184,8.078227,551.932003,0.256269,...,0.000220,0.000038,4.852768e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,547.315987,-1.343191,1.804162,299554.789688,806.497248,-0.908726,650437.811788,0.825784,550.669163,-1.262840,...,0.000058,0.000037,3.414065e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,554.966041,-1.277939,1.633127,307987.307133,813.088753,-6.370867,661113.320100,40.587943,552.662723,-0.652898,...,0.000004,0.000019,1.436410e-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,554.111798,-0.854243,0.729731,307039.885019,810.995302,-2.093451,657713.379057,4.382539,552.448883,-0.213841,...,0.000335,0.000016,1.119772e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224,553.453749,-0.658049,0.433028,306311.052626,815.053154,4.057853,664311.643996,16.466168,552.643977,0.195094,...,-0.000573,0.000021,3.285238e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225,551.820319,-1.633431,2.668096,304505.664025,808.053376,-6.999778,652950.258208,48.996896,552.455372,-0.188605,...,-0.000682,0.000028,4.651786e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
